# [LVV-T2732] - StarTracker Pointing and Tracking Test - Slew and Settle - TMA Tracking Jitter Validation

**Requirements:**
 - All the MT components should be enabled.
 - Need the `notebooks_vandv` module installed.

Please, see the [README] file for the requirements to run this notebook.  

[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[LVV-T2730]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2730

## Setup

In [152]:
test_case = "LVV-T2732"
test_exec = "LVV-EXXXX"

# az_grid = [225, 135, 45, -45]
# el_grid = [25, 35, 45, 55, 65, 75]

az_grid = [135, 45, -45, -135]
el_grid = [25, 35, 45, 55, 65, 75]

# 101 - Wide Camera
# 102 - Narrow Camera
# 103 - Fast Camera (DIMM)
camera_sal_indexes = [101, 102, 103]
exposure_times = [5., 4., 6.]  # s
base_msg = f"{test_case} {test_exec}:"

number_of_exposures = 5
n_offsets = 5
offset_size = 3.5 # degrees
track_time = 60. 

sleep_time= 4. #Time sleeping in each offset position

### Prepare Notebook

Start with importing libraries:

In [153]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import numpy as np
import sys
import yaml

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.MTDome import SubSystemId
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control import RotType
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control.generic_camera import GenericCamera

from lsst.sitcom import vandv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
Print out the execution info for future reference.

In [3]:
exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by isotuela on 2023-03-24T21:50:49.685.
  Running in yagan02 at summit



---
Setup the logger, the Domain, and the remote for the telescope control.

In [5]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)
log = logging.getLogger(f"{test_case}")

In [7]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

My user_host is 'isotuela@nb-isotuela'



---
The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [8]:
script = salobj.Controller("Script", index=199)
await script.start_task

script.log.info("Testing Log")

Instanciate the MTCS.

In [9]:
mtcs = MTCS(domain=domain, log=script.log)
vandv.logger.add_filter_to_mtcs()
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

--- 
Initialize the GenericCameras.  
You might need to modify the cell below. 

In [46]:
camera_list = []

for index in camera_sal_indexes:

    gencam = GenericCamera(domain=domain, index=index, log=script.log)
    await gencam.start_task
    #await gencam.enable()

    camera_list.append(gencam)

In [47]:
print(camera_list)

[<lsst.ts.observatory.control.generic_camera.GenericCamera object at 0x7f896ec406a0>, <lsst.ts.observatory.control.generic_camera.GenericCamera object at 0x7f896f263c10>, <lsst.ts.observatory.control.generic_camera.GenericCamera object at 0x7f896f373730>]


### Check CSC Status

In [154]:
# Ignore (set check to False) the following components:
list_of_cscs = [
    "mtaos",
    "mtdome",
    "mtdometrajectory",
    "mthexapod_1",
    "mthexapod_2",
    "mtm1m3",
    "mtm2",
    "mtrotator",
]

for csc in list_of_cscs:
    script.log.info(f"Setting {csc} check to False")
    setattr(mtcs.check, csc, False)

In [13]:
await mtcs.set_state(
    salobj.State.DISABLED, 
    components=[
        "mtmount", 
        #"mtrotator",
        #"mtptg",
    ]
)

In [14]:
await mtcs.set_state(
    salobj.State.ENABLED, 
    components=[
        "mtmount",
        # "mtrotator", # Enable this only if using the Rotator Hardware
        "mtptg",
    ]
)

In [15]:
## Increase MTMount Log Verbosity
await mtcs.rem.mtmount.cmd_setLogLevel.set_start(level=15)

In [155]:
## Comment this line if you want the CCW to follow the Rotator
await mtcs.enable_ccw_following()

In [158]:
## Home MTMount Main Axes
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

In [159]:
await mtcs.assert_liveliness()

## Enable this only if all the MT CSCs are enabled
# await mtcs.assert_all_enabled()

In [160]:
for cam in camera_list:
    await cam.assert_liveliness()
    await cam.assert_all_enabled()

### Confirm take image each camera

In [ ]:
reason = "header_check"

await camera_list[0].take_object(exptime=1., reason=reason)
await camera_list[1].take_object(exptime=1., reason=reason)
await camera_list[2].take_object(exptime=1., reason=reason)

In [115]:
await camera_list[2].take_object(exptime=1., reason="Sync")

[2023032400077]

### Confirm Take Images in Sync

In [ ]:
res = (number_of_exposures) * len(camera_list)
for n in range(number_of_exposures):
    tasks = [asyncio.create_task(cam.take_object(exptime, reason=base_msg[:-1])) 
             for (cam, exptime) in zip(camera_list, exposure_times)]
    await asyncio.gather(*tasks)

### Other preparation

- LVV-T2713 (1.0) Establish TMA - StarTracker Axis Angle Reference

## Helper Functions

In [23]:
async def wait_for_dome_in_position():
    """Wait until the dome is in position"""
    await asyncio.sleep(20)
    azMotion = await mtcs.rem.mtdome.evt_azMotion.aget()

    while not azMotion.inPosition:
        azMotion = await mtcs.rem.mtdome.evt_azMotion.aget()
        await asyncio.sleep(5.)
        
    if azMotion.state == 1.:
        await mtcs.rem.mtdome.cmd_exitFault.set_start()

In [162]:
async def take_images_in_sync(_camera_list, _exposure_times, _number_of_exposures, _reason, total_time): 
    """
    Take images in sync, which means keeping the images ID the same. 
    This will increase overhead on the camera with shorter exposure time.
    
    Parameters
    ----------
    _camera_list : list of `GenericCamera`
        A list containing the `GenericCamera` for each Camera.
    _exposure_times : list of float 
        A list containing the exposure time used on each camera.
    _reason : str 
        Reason that goes to the metadata in each image.
    _number_of_exposures : float
        Total number of exposures for each camera.
    total_time : float
        Minimum time we should spend taking images (to keep tracking in a fixed position).
    """
    assert len(_camera_list) == len(_exposure_times)

    wait_time = asyncio.create_task(asyncio.sleep(total_time))
    
    for n in range(_number_of_exposures):
        tasks = [asyncio.create_task(cam.take_object(exptime, reason=_reason)) 
                 for (cam, exptime) in zip(_camera_list, _exposure_times)]
        
        # Wait until all the tasks are complete
        await asyncio.gather(*tasks)
        
    await wait_time

## Data Acquisition

### Find target at defined az and el

In [ ]:
az_target = 
el_target = 

In [ ]:
# Center fast camera

In [137]:
target = await mtcs.find_target(az=az_target,el=el_target, mag_limit=2, radius=10)
print(target)

In [139]:
radec = await mtcs.slew_object(target, rot=0, rot_type=RotType.Physical)
print(f'Slew to target. The radec is {radec}')

(<ICRS Coordinate: (ra, dec) in deg
    (83.18256708, -17.82228917)>, <Angle 0. deg>)


In [140]:
azel= mtcs.azel_from_radec(radec[0].ra, radec[0].dec)
az = azel.az
el = azel.alt
print(f'The azel of the target is {az.deg}, {el.deg}')

277.92711783037714 49.52859260475941


###  Move Dome to the required AZ

In [141]:
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az, velocity=0)

In [142]:
# Wait untin dome settles and engage breaks. 
sub_system_ids = SubSystemId.AMCS
await mtcs.rem.mtdome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)

In [ ]:
await mtcs.rem.mtdome.cmd_exitFault.set_start()

### Offset the telescope to center bright start on Fast Camera

In [143]:
# Standard fixed offset 
await mtcs.offset_azel(az=208, el=508)

In [145]:
# Additional offset? 
await mtcs.offset_azel(az=0, el=60)

### Take images inSync

In [49]:
number_of_exposures = 1

In [146]:
for n in range(number_of_exposures):
    tasks = [asyncio.create_task(cam.take_object(exptime, reason=base_msg[:-1])) 
             for (cam, exptime) in zip(camera_list, exposure_times)]
    await asyncio.gather(*tasks)

### Declare original ra and dec for the centered target (Target radec plus add positional offsets) 

In [147]:
ra_orig_plusoffset = mtcs.rem.mtptg.tel_mountPosition.get().ra
dec_orig_plusoffset = mtcs.rem.mtptg.tel_mountPosition.get().declination
print(f'The original radec for the centered target {ra_orig_plusoffset}, {dec_orig_plusoffset}')
# await mtcs.slew_icrs(ra=ra_orig*u.deg,dec=dectest*u.deg,rot=0,rot_type=RotType.Physical)

83.30608184452312 -17.943110689618248


### Random 3.5 deg offset 

In [151]:
script.log.info(f"{base_msg} Streaming  - Start")
# Wait sleep time
await asyncio.sleep(sleep_time)

script.log.info(f"{base_msg} Apply random offsets - Start")

random_angle = 2 * np.pi * np.random.rand()
offset_dec = offset_size * np.cos(random_angle)

sign = 1 if np.random.rand() < 0.5 else -1
offset_ra = sign * np.sqrt(offset_size ** 2 - offset_dec ** 2) / np.cos(radec[0].dec.rad)

script.log.info(f"{base_msg} Offset # - ra = {offset_ra:.5f}, dec = {offset_dec:.5f} - Start")

# Random offset
await mtcs.slew_icrs(
    ra=ra_orig_plusoffset*u.deg + offset_ra * u.deg, 
    dec=dec_orig_plusoffset*u.deg + offset_dec * u.deg,
    rot=0, 
    rot_type=RotType.Physical
)
script.log.info(f"{base_msg} Offset #- ra = {offset_ra:.5f}, dec = {offset_dec:.5f} - Done")

await asyncio.sleep(sleep_time)

#Back to the original position
script.log.info(f"{base_msg} Slew back to original sky position - Start")
await mtcs.slew_icrs(ra=ra_orig_plusoffset*u.deg, dec=dec_orig_plusoffset*u.deg, rot=0, rot_type=RotType.Physical)
script.log.info(f"{base_msg} Slew back to original sky position - Done")

await asyncio.sleep(sleep_time)
script.log.info(f"{base_msg} Streaming - Done")
    
script.log.info(f"{base_msg} Apply random offset - Done")

## Dome Commands

The following commands allow to control the Dome. However, make sure it is in ENABLED state. Use LOVE to change its state.

### Move to position

In [39]:
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=81, velocity=0)

### Stop the Dome

In [ ]:
sub_system_ids = SubSystemId.AMCS
await dome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)

### Recover from fault when stopping

In [164]:
await mtcs.rem.mtdome.cmd_exitFault.set_start()